<a href="https://colab.research.google.com/github/s-kamireddy/AI_HW_6/blob/main/AI_HW_6_Part4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [121]:
#iterate thru

In [122]:
ratings = pd.read_csv('u.data',sep = '\t', names=['userId', 'movieId', 'rating', 'timestamp'])
ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.head()


,userId,movieId,rating,timestamp,date
0,196,242,3,881250949,1997-12-04 15:55:49
1,186,302,3,891717742,1998-04-04 19:22:22
2,22,377,1,878887116,1997-11-07 07:18:36
3,244,51,2,880606923,1997-11-27 05:02:03
4,166,346,1,886397596,1998-02-02 05:33:16


In [123]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   userId     100000 non-null  int64         
 1   movieId    100000 non-null  int64         
 2   rating     100000 non-null  int64         
 3   timestamp  100000 non-null  int64         
 4   date       100000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 3.8 MB


In [124]:
# Define column names based on the documentation
item_columns = ['item_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL'] + [f'genre_{i}' for i in range(19)]
movies = pd.read_csv('u.item', sep='|', header=None, names=item_columns, encoding='latin-1')
movies.drop(columns=['video_release_date', 'IMDb_URL', 'release_date', 'movie_title'], inplace=True)
movies.head()

,item_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [125]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   item_id   1682 non-null   int64
 1   genre_0   1682 non-null   int64
 2   genre_1   1682 non-null   int64
 3   genre_2   1682 non-null   int64
 4   genre_3   1682 non-null   int64
 5   genre_4   1682 non-null   int64
 6   genre_5   1682 non-null   int64
 7   genre_6   1682 non-null   int64
 8   genre_7   1682 non-null   int64
 9   genre_8   1682 non-null   int64
 10  genre_9   1682 non-null   int64
 11  genre_10  1682 non-null   int64
 12  genre_11  1682 non-null   int64
 13  genre_12  1682 non-null   int64
 14  genre_13  1682 non-null   int64
 15  genre_14  1682 non-null   int64
 16  genre_15  1682 non-null   int64
 17  genre_16  1682 non-null   int64
 18  genre_17  1682 non-null   int64
 19  genre_18  1682 non-null   int64
dtypes: int64(20)
memory usage: 262.9 KB


In [126]:
users = pd.read_csv('u.user', sep='|', header=None, names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])
users.head()


,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [127]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     943 non-null    int64 
 1   age         943 non-null    int64 
 2   gender      943 non-null    object
 3   occupation  943 non-null    object
 4   zip_code    943 non-null    object
dtypes: int64(2), object(3)
memory usage: 37.0+ KB


In [128]:
users['gender'] = users['gender'].map({'F': 1, 'M': 0})
users['occupation'] = users['occupation'].astype('category').cat.codes
users['zip_code'] = users['zip_code'].astype('category').cat.codes
users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,0,19,622
1,2,53,1,13,689
2,3,23,0,20,270
3,4,24,0,19,331
4,5,33,1,13,133


In [129]:
data = ratings.merge(movies, left_on='movieId', right_on='item_id')
data = data.merge(users, left_on='userId', right_on='user_id')
data.drop(columns=['item_id'], inplace=True)
data.head()


,userId,movieId,rating,timestamp,date,genre_0,genre_1,genre_2,genre_3,genre_4,...,genre_14,genre_15,genre_16,genre_17,genre_18,user_id,age,gender,occupation,zip_code
0,196,242,3,881250949,1997-12-04 15:55:49,0,0,0,0,0,...,0,0,0,0,0,196,49,0,20,415
1,186,302,3,891717742,1998-04-04 19:22:22,0,0,0,0,0,...,0,0,1,0,0,186,39,1,6,0
2,22,377,1,878887116,1997-11-07 07:18:36,0,0,0,0,1,...,0,0,0,0,0,22,25,0,20,311
3,244,51,2,880606923,1997-11-27 05:02:03,0,0,0,0,0,...,1,0,0,1,1,244,28,0,19,591
4,166,346,1,886397596,1998-02-02 05:33:16,0,0,0,0,0,...,0,0,0,0,0,166,47,0,3,420


In [130]:
data.drop(columns=['date'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   userId      100000 non-null  int64
 1   movieId     100000 non-null  int64
 2   rating      100000 non-null  int64
 3   timestamp   100000 non-null  int64
 4   genre_0     100000 non-null  int64
 5   genre_1     100000 non-null  int64
 6   genre_2     100000 non-null  int64
 7   genre_3     100000 non-null  int64
 8   genre_4     100000 non-null  int64
 9   genre_5     100000 non-null  int64
 10  genre_6     100000 non-null  int64
 11  genre_7     100000 non-null  int64
 12  genre_8     100000 non-null  int64
 13  genre_9     100000 non-null  int64
 14  genre_10    100000 non-null  int64
 15  genre_11    100000 non-null  int64
 16  genre_12    100000 non-null  int64
 17  genre_13    100000 non-null  int64
 18  genre_14    100000 non-null  int64
 19  genre_15    100000 non-null  int64
 20  genre

In [131]:
class MovieLensDataset(Dataset):
  def __init__(self, data):
    self.data = data

    self.user_map = {user_id: idx for idx, user_id in enumerate(data['userId'].unique())}
    self.item_map = {item_id: idx for idx, item_id in enumerate(data['movieId'].unique())}

    self.user_ids = torch.tensor(self.data['userId'].map(self.user_map).values, dtype=torch.long)
    self.item_ids = torch.tensor(self.data['movieId'].map(self.item_map).values, dtype=torch.long)
    self.ratings = torch.tensor(self.data['rating'].values, dtype=torch.float32)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return {'userId': self.user_ids[idx], 'movieId': self.item_ids[idx], 'rating': self.ratings[idx]}

In [132]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
train = MovieLensDataset(train)
test = MovieLensDataset(test)
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=False)

Implementing Matrix Factorization

In [153]:
class RecommenderSystem(nn.Module):
  def __init__(self, num_users, num_items, embedding_dim):
    super(RecommenderSystem, self).__init__()
    self.user_embedding = nn.Embedding(num_users, embedding_dim)
    self.item_embedding = nn.Embedding(num_items, embedding_dim)

  def forward(self, user_input, item_input):
    user_embedded = self.user_embedding(user_input)
    item_embedded = self.item_embedding(item_input)
    output = torch.sum(torch.mul(user_embedded, item_embedded), dim = 1)
    return output


In [154]:
model = RecommenderSystem(num_users=data['userId'].nunique(), num_items=data['movieId'].nunique(), embedding_dim=50)

In [155]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RecommenderSystem(
  (user_embedding): Embedding(943, 50)
  (item_embedding): Embedding(1682, 50)
)

In [156]:
for epoch in range(10):
  model.train()
  for idx, batch in enumerate(train_loader):
    user_input, item_input, rating = batch['userId'], batch['movieId'], batch['rating']



    optimizer.zero_grad()
    prediction = model(user_input, item_input)
    loss = loss_fn(prediction, rating)
    loss.backward()
    optimizer.step()



Evaluating performance with r^2 and MSE loss
source: https://medium.com/data-science/metrics-of-recommender-systems-cde64042127a

In [157]:
#evaluate model
model.eval()

r_squared = []
losses = []
mapks = []
batch_size = 64
predictions = []
targets = []

for idx, batch in enumerate(test_loader):
  user_input, item_input, rating = batch['userId'], batch['movieId'], batch['rating']
  prediction = model(user_input, item_input)

  predictions.extend(prediction.detach().numpy())
  targets.extend(rating.detach().numpy())

  loss = loss_fn(prediction, rating)
  r_squared.append(1 - (loss.item() / rating.var()))
  losses.append(loss.item())



print(f'Average loss: {np.mean(losses)/batch_size}')
print(f'RMSE: {np.sqrt(mean_squared_error(targets, predictions))}')
print(f'R-squared: {np.mean(r_squared)}')
print(f'first 10 predictions: {predictions[:10]}')
print(f'first 10 targets: {targets[:10]}')



Average loss: 0.4660756934565096
RMSE: 5.459355234276726
R-squared: -23.175045013427734
first 10 predictions: [np.float32(-1.3615773), np.float32(3.1838782), np.float32(0.68508434), np.float32(3.0849068), np.float32(3.47143), np.float32(1.5051731), np.float32(1.6106794), np.float32(2.1328254), np.float32(1.2954607), np.float32(-0.6524959)]
first 10 targets: [np.float32(4.0), np.float32(3.0), np.float32(4.0), np.float32(2.0), np.float32(2.0), np.float32(3.0), np.float32(5.0), np.float32(4.0), np.float32(3.0), np.float32(4.0)]


Implementing collaborative filtering

In [172]:
class RecSysModel(nn.Module):
    def __init__(self, n_users, n_movies):
        super().__init__()
        # trainable lookup matrix for shallow embedding vectors
        self.user_embed = nn.Embedding(n_users, 50)
        self.movie_embed = nn.Embedding(n_movies, 50)
        # user, movie embedding concat
        self.out = nn.Linear(100, 1)


    def forward(self, users, movies, ratings=None):
        user_embeds = self.user_embed(users)
        movie_embeds = self.movie_embed(movies)
        output = torch.cat([user_embeds, movie_embeds], dim=1)

        output = self.out(output)

        return output

In [173]:
model = RecSysModel(n_users=data['userId'].nunique(), n_movies=data['movieId'].nunique())

In [174]:
for epoch in range(10):
  model.train()
  for idx, batch in enumerate(train_loader):
    user_input, item_input, rating = batch['userId'], batch['movieId'], batch['rating']



    optimizer.zero_grad()
    prediction = model(user_input, item_input)
    loss = loss_fn(prediction, rating)
    loss.backward()
    optimizer.step()



In [175]:
#eval with same metrics
#evaluate model
model.eval()

r_squared = []
losses = []
mapks = []
batch_size = 64
predictions = []
targets = []

for idx, batch in enumerate(test_loader):
  user_input, item_input, rating = batch['userId'], batch['movieId'], batch['rating']
  prediction = model(user_input, item_input)

  predictions.extend(prediction.detach().numpy())
  targets.extend(rating.detach().numpy())

  loss = loss_fn(prediction, rating)
  r_squared.append(1 - (loss.item() / rating.var()))
  losses.append(loss.item())




print(f'Average loss: {np.mean(losses)/batch_size}')
print(f'RMSE: {np.sqrt(mean_squared_error(targets, predictions))}')
print(f'R-squared: {np.mean(r_squared)}')
print(f'first 10 predictions: {predictions[:10]}')
print(f'first 10 targets: {targets[:10]}')

Average loss: 0.2118302015736461
RMSE: 3.685523218383894
R-squared: -10.01546859741211
first 10 predictions: [array([0.5531127], dtype=float32), array([-0.35298595], dtype=float32), array([-0.4506967], dtype=float32), array([-0.44744623], dtype=float32), array([-0.47373283], dtype=float32), array([0.2948361], dtype=float32), array([-0.5983819], dtype=float32), array([-0.54363996], dtype=float32), array([-0.81055576], dtype=float32), array([0.49172026], dtype=float32)]
first 10 targets: [np.float32(4.0), np.float32(3.0), np.float32(4.0), np.float32(2.0), np.float32(2.0), np.float32(3.0), np.float32(5.0), np.float32(4.0), np.float32(3.0), np.float32(4.0)]


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


The collaborative filtering model performs better in terms of MSE and R^2 error as the linear layer allows for a more precise fit